Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# function for accuracy calculation
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Model
Using l2 regularization for logistic model

In [5]:
batch_size = 128
l_rate = 0.5 # learning rate
lambda_l2 = 0.01 # lambda value for l2 regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += lambda_l2 * tf.nn.l2_loss(weights)  # include l2 norm to the loss

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(l_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.499214
Minibatch accuracy: 7.0%
Validation accuracy: 11.0%
Minibatch loss at step 500: 0.996056
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.959393
Minibatch accuracy: 79.7%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 0.761216
Minibatch accuracy: 78.9%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 0.706408
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.768983
Minibatch accuracy: 78.1%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.821031
Minibatch accuracy: 79.7%
Validation accuracy: 82.5%
Test accuracy: 88.0%


### Neural Network Model
Using l2 regularization for Neural Network Model

In [7]:
batch_size = 128
num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(h1, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  loss += lambda_l2 * (tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)) # include the regularization to loss
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(l_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_logits = tf.matmul(valid_h1, weights_2) + biases_2  
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_logits = tf.matmul(test_h1, weights_2) + biases_2 
  test_prediction = tf.nn.softmax(test_logits)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3530.227051
Minibatch accuracy: 9.4%
Validation accuracy: 36.7%
Minibatch loss at step 500: 21.309956
Minibatch accuracy: 85.9%
Validation accuracy: 85.2%
Minibatch loss at step 1000: 0.984570
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 0.767349
Minibatch accuracy: 82.0%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 0.722803
Minibatch accuracy: 83.6%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.753445
Minibatch accuracy: 82.8%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 0.844719
Minibatch accuracy: 79.7%
Validation accuracy: 84.6%
Test accuracy: 89.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# reduce the training data to 1000
reduced_train = train_dataset[:1000,:] 
reduced_labels = train_labels[:1000,:]

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (reduced_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = reduced_train[offset:(offset + batch_size), :]
    batch_labels = reduced_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3415.769775
Minibatch accuracy: 8.6%
Validation accuracy: 29.7%
Minibatch loss at step 500: 21.053179
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.536991
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 0.348700
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 0.331309
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 0.415909
Minibatch accuracy: 96.1%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.319706
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Test accuracy: 85.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128
num_nodes = 1024
l_rate = 0.05 # learning rate
lambda_l2 = 0.01 # lambda value for l2 regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # a placeholder to set the drop out rate later
  keep_prob = tf.placeholder(tf.float32) 
  # use dropout for regularization
  h1_drop = tf.nn.dropout(h1, keep_prob)

  logits = tf.matmul(h1_drop, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
  loss += lambda_l2 * (tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))  # include the regularization to loss

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_logits = tf.matmul(valid_h1, weights_2) + biases_2  
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_logits = tf.matmul(test_h1, weights_2) + biases_2 
  test_prediction = tf.nn.softmax(test_logits)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3619.483154
Minibatch accuracy: 12.5%
Validation accuracy: 37.9%
Minibatch loss at step 500: 21.521646
Minibatch accuracy: 76.6%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 1.060723
Minibatch accuracy: 79.7%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 0.843853
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 2000: 0.799142
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.833736
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.956461
Minibatch accuracy: 76.6%
Validation accuracy: 84.0%
Test accuracy: 89.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Using two hidden layers with 1024 and 512 nodes
Applies both dropout and l2 regularization to prevent overfitting.

Use expoential decay function to decrease the learning rate through the training process.

In [14]:
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 512

lambda_l2 = 0.00001
starter_learning_rate = 0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variable - initialize the weight using Xavier initialization in case of a network start too small.
  # https://discussions.udacity.com/t/assignment-3-loss-becomes-nan-when-using-more-than-1-hidden-layer/164099/11 
  # http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization

  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1], \
                        stddev= 2.0 /float(image_size * image_size))) # 728 * 1024
  biases_1 = tf.Variable(tf.constant(0.1, shape= [hidden_nodes_1]))
    
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], 2.0 /float(hidden_nodes_1))) # 1024 * 512
  biases_2 = tf.Variable(tf.constant(0.1, shape= [hidden_nodes_2]))
    
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_nodes_2, num_labels], 2.0 /float(hidden_nodes_2))) # 512 * 10
  biases_3 = tf.Variable(tf.constant(0.1, shape= [num_labels]))
    
  # Training computation using two hidden layers
  keep_prob = tf.placeholder("float")

  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1) #hidden layer 1
  hidden_1_drop = tf.nn.dropout(hidden_1, keep_prob)
  hidden_2 = tf.nn.relu(tf.matmul(hidden_1_drop, weights_2) + biases_2) #hidden layer 2
  hidden_2_drop = tf.nn.dropout(hidden_2, keep_prob)
  logits = tf.matmul(hidden_2_drop, weights_3) + biases_3 #output layer
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  #calculate the l2 regularization for all three weights  
  regularization = lambda_l2 * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
     tf.nn.l2_loss(weights_3)) 
  #loss += regularization
  
  # use exponential decay for adjusting the learning rate.
  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           2000, 0.96, staircase=True)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_2 = tf.nn.relu(tf.matmul(valid_1, weights_2) + biases_2)  
  valid_prediction = tf.nn.softmax(tf.matmul(valid_2, weights_3) + biases_3)
    
  test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_2 = tf.nn.relu(tf.matmul(test_1, weights_2) + biases_2) 
  test_prediction = tf.nn.softmax(tf.matmul(test_2, weights_3) + biases_3)

In [15]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 85.073181
Minibatch accuracy: 11.7%
Validation accuracy: 33.6%
Minibatch loss at step 2000: 2.873104
Minibatch accuracy: 77.3%
Validation accuracy: 86.0%
Minibatch loss at step 4000: 2.909473
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Minibatch loss at step 6000: 1.123646
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Minibatch loss at step 8000: 2.291084
Minibatch accuracy: 85.9%
Validation accuracy: 87.6%
Minibatch loss at step 10000: 1.089586
Minibatch accuracy: 84.4%
Validation accuracy: 87.9%
Minibatch loss at step 12000: 2.581045
Minibatch accuracy: 84.4%
Validation accuracy: 88.3%
Minibatch loss at step 14000: 3.109596
Minibatch accuracy: 82.8%
Validation accuracy: 88.3%
Minibatch loss at step 16000: 1.834740
Minibatch accuracy: 83.6%
Validation accuracy: 88.7%
Minibatch loss at step 18000: 1.008777
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 20000: 2.336281
Minibatch accuracy: 85.9%
Validation